<a href="https://colab.research.google.com/github/alexxandert/Stock-Market-Performance/blob/main/Stock_Market_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime # datetime is a module for manipulating dates and times.

#This sets start_date to 3 months ago and end_date to the current date.
start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG'] #stock tickers

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

#This combines all the separate data into one DataFrame with multi-level indexing
df = pd.concat(df_list, keys = tickers , names = ['Ticker' , 'Date'])
print(df.head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-05-26  173.320007  175.770004  173.110001  175.429993  175.193420   
       2023-05-30  176.960007  178.990005  176.570007  177.300003  177.060898   
       2023-05-31  177.330002  179.350006  176.759995  177.250000  177.010971   
       2023-06-01  177.699997  180.119995  176.929993  180.089996  179.847137   
       2023-06-02  181.029999  181.779999  179.259995  180.949997  180.705978   

                     Volume  
Ticker Date                  
AAPL   2023-05-26  54835000  
       2023-05-30  55964400  
       2023-05-31  99625300  
      

In [ ]:
df = df.reset_index()  #Reset the multi-level index to columns:
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-05-26  173.320007  175.770004  173.110001  175.429993   
1   AAPL 2023-05-30  176.960007  178.990005  176.570007  177.300003   
2   AAPL 2023-05-31  177.330002  179.350006  176.759995  177.250000   
3   AAPL 2023-06-01  177.699997  180.119995  176.929993  180.089996   
4   AAPL 2023-06-02  181.029999  181.779999  179.259995  180.949997   

    Adj Close    Volume  
0  175.193420  54835000  
1  177.060898  55964400  
2  177.010971  99625300  
3  179.847137  68901800  
4  180.705978  61945900  


In [ ]:
import plotly.express as px #plotly is a powerful visualization library
fig = px.line(df, x='Date',
              y='Close',
              color='Ticker',
              title="Stock Market Performance for the Last 3 Months")
fig.show()

In [ ]:
fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

In [ ]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

# df.groupby('Ticker')['Close']: This groups the DataFrame df by the unique values in the 'Ticker' column and then selects the 'Close' column (which likely contains the closing prices for stocks).
# .rolling(window=10).mean(): This calculates a rolling mean (also known as a moving average) over a window of 10 days. In other words, for each day, it computes the average closing price of the current day and the 9 preceding days.
# .reset_index(0, drop=True): After computing the rolling mean, the resulting object has a multi-level index (with 'Ticker' as one of the levels). This line of code resets the first level of the index and drops it, so that the result aligns properly with the original df.

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

#  df.groupby('Ticker'): This again groups the DataFrame by the 'Ticker' values. The resulting object allows you to work with each group of rows that share the same ticker.
# for ticker, group in ...: For each group, ticker will be the unique value of the 'Ticker' (i.e., the stock ticker like 'AAPL', 'MSFT', etc.), and group will be a sub-DataFrame containing all the rows for that particular ticker.



Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
58  177.156001  184.269001
59  176.899001  183.449501
60  177.192000  182.780501
61  177.033000  181.938501
62  177.115001  181.077501

[63 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
189         NaN         NaN
190         NaN         NaN
191         NaN         NaN
192         NaN         NaN
193         NaN         NaN
..          ...         ...
247  130.108000  129.965000
248  129.893001  130.310000
249  130.199002  130.487500
250  130.220001  130.515000
251  130.272002  130.399001

[63 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
63          NaN         NaN
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
..          ...         ...
121  321.376001  327.79

In [ ]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'],
                  title=f"{ticker} Moving Averages")
    fig.show()

In [ ]:
# Volatility is a measure of how much the price of a security varies over time.
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility',
              color='Ticker',
              title='Volatility of All Companies')
fig.show()

In [ ]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
#df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']]: This is using the loc function of the DataFrame to filter and select rows where the 'Ticker' column value is 'AAPL'. From those rows, it's selecting the 'Date' and 'Close' columns.
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')


# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT',
                 trendline='ols',    #trendline='ols': Adds an ordinary least squares regression trendline to the scatter plot. This helps visualize the linear relationship between Apple and Microsoft's stock prices.
                 title='Correlation between Apple and Microsoft')
fig.show()
